In [4]:
!pip install neuralprophet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 37.6 MB/s eta 0:00:00


In [70]:
# downloading data
from google.colab import drive
import pandas as pd

# NVIDIA data_source: https://www.nasdaq.com/market-activity/stocks/nvda/historical
# INV NVIDIA data source: https://www.nasdaq.com/market-activity/etf/nvdq/historical?page=1&rows_per_page=10&timeline=y10
#
data_folder = "/content/drive/My Drive/CMPE257ProjectData/"
drive.mount('/content/drive')

NVIDIA_path = "NVIDIA.csv"
INV_NVIDIA_path = "InvNVIDIA.csv"

NVIDIA_df = pd.read_csv(data_folder + NVIDIA_path)
INV_NVIDIA_df = pd.read_csv(data_folder + INV_NVIDIA_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
print(NVIDIA_df.head())
print(INV_NVIDIA_df.head())

         Date Close/Last     Volume     Open     High      Low
0  12/13/2024    $134.25  231514900  $138.94  $139.60  $132.54
1  12/12/2024    $137.34  159211400  $137.08  $138.44  $135.80
2  12/11/2024    $139.31  184905200  $137.36  $140.17  $135.21
3  12/10/2024    $135.07  210020900  $139.01  $141.82  $133.79
4  12/09/2024    $138.81  189308600  $138.97  $139.95  $137.13
         Date  Close/Last    Volume  Open    High    Low
0  12/13/2024       3.530  65500680  3.29  3.6199  3.260
1  12/12/2024       3.380  39795880  3.40  3.4500  3.325
2  12/11/2024       3.280  62519890  3.38  3.4999  3.240
3  12/10/2024       3.500  57787980  3.32  3.5644  3.180
4  12/09/2024       3.325  45752180  3.31  3.3998  3.270


In [93]:
print(len(NVIDIA_df))
print(len(INV_NVIDIA_df))

1258
291


In [72]:
NVIDIA_df["Open"] = NVIDIA_df["Open"].str.replace('$','').astype(float)
NVIDIA_df["Close/Last"] = NVIDIA_df["Close/Last"].str.replace('$','').astype(float)
NVIDIA_df["High"] = NVIDIA_df["High"].str.replace('$','').astype(float)
NVIDIA_df["Low"] = NVIDIA_df["Low"].str.replace('$','').astype(float)

NVIDIA_df.head()

,Date,Close/Last,Volume,Open,High,Low
0,12/13/2024,134.25,231514900,138.94,139.60,132.54
1,12/12/2024,137.34,159211400,137.08,138.44,135.80
2,12/11/2024,139.31,184905200,137.36,140.17,135.21
3,12/10/2024,135.07,210020900,139.01,141.82,133.79
4,12/09/2024,138.81,189308600,138.97,139.95,137.13


In [73]:
for col in NVIDIA_df.columns:
  print(f'column name: {col}, type: {NVIDIA_df[col].dtype} ')
print('\n')
for col in INV_NVIDIA_df.columns:
  print(f'column name: {col}, type: {NVIDIA_df[col].dtype} ')

column name: Date, type: object 
column name: Close/Last, type: float64 
column name: Volume, type: int64 
column name: Open, type: float64 
column name: High, type: float64 
column name: Low, type: float64 


column name: Date, type: object 
column name: Close/Last, type: float64 
column name: Volume, type: int64 
column name: Open, type: float64 
column name: High, type: float64 
column name: Low, type: float64 


In [76]:
def preprocess(data):
  proc_data = data.copy()
  proc_data['Date'] = pd.to_datetime(proc_data['Date'])
  proc_data.rename(columns={'Close/Last': 'Close'}, inplace=True)
  proc_data = proc_data.iloc[::-1].reset_index(drop=True)
  # normalize ???
  return proc_data

proc_NVIDIA_df = preprocess(NVIDIA_df)
#proc_INV_NVIDIA_df = preprocess(INV_NVIDIA_df)

proc_NVIDIA_df.head()

,Date,Close,Volume,Open,High,Low
0,2019-12-16,5.6288,321307360,5.6475,5.7016,5.6193
1,2019-12-17,5.7073,323263920,5.6475,5.7300,5.6380
2,2019-12-18,5.7378,344278760,5.7423,5.7985,5.7338
3,2019-12-19,5.8865,453106800,5.7725,5.9000,5.7675
4,2019-12-20,5.9843,603384400,5.9533,6.0098,5.9339


ADF Statistic: -10.787950672822788
p-value: 2.1698217850849596e-19


In [112]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, train_test_split

def get_features(data):
  X = data.copy()

  # Date features:
  X['year'] = X['Date'].dt.year
  X['month'] = X['Date'].dt.month
  X['day'] = X['Date'].dt.day
  X['dayofweek'] = X['Date'].dt.dayofweek

  y1 = X['Open']
  y2 = X['Close']
  y3 = X['High']
  y4 = X['Low']
  X = X.drop(columns=['Open', 'Close', 'High', 'Low', 'Date', 'Volume'])

  return X, y1, y2, y3, y4

def data_split(X, y):
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
  return X_train, X_test, y_train, y_test

def train_model(X_train, y_train):
  parameters = {
      'n_estimators': [10, 25, 50 , 100, 200],
      'max_depth': [5, 10, 20, None],
      'min_samples_leaf': [1, 2, 4, 8]
  }

  model = RandomizedSearchCV(
      estimator=RandomForestRegressor(),
      param_distributions=parameters,
      n_iter=20,
      cv=3
  )

  model.fit(X_train, y_train)

  print(model.best_params_)

  return model.best_estimator_

NVIDIA_features, open, close, high, low = get_features(proc_NVIDIA_df)

X_train, X_test, y_train, y_test = data_split(NVIDIA_features, open)

model = train_model(X_train, y_train)

y_pred = model.predict(X_test)
print(f'MSE: {mean_squared_error(y_test, y_pred)}')



#INV_NVIDIA_features = get_features(proc_INV_NVIDIA_df)

{'n_estimators': 200, 'min_samples_leaf': 1, 'max_depth': None}
MSE: 5.078717304288543


In [113]:
def generate_predictions(model, date):
  curr_day = date
  X_pred = pd.DataFrame(columns=['year', 'month', 'day', 'dayofweek'])
  for i in range(7):
    curr_day = curr_day + pd.Timedelta(days=1)
    if curr_day.dayofweek == 5 or curr_day.dayofweek == 6:
      continue

    X_pred.loc[i] = [curr_day.year, curr_day.month, curr_day.day, curr_day.dayofweek]

  print(model.predict(X_pred))
  return

generate_predictions(model, proc_NVIDIA_df['Date'].iloc[-1])


[138.725275 138.699875 138.950975 139.8199   140.2498  ]


In [85]:
from neuralprophet import NeuralProphet


def train_model(data):
  X_train = data.iloc[:-5]
  X_test = data.iloc[-5:]

  open_df = X_train[["Date", "Open"]]
  open_df.columns = ["ds", "y"]

  open_model = NeuralProphet()
  open_model.fit(open_df, freq="D")


  close_df = X_train[["Date", "Close"]]
  close_df.columns = ["ds", "y"]

  close_model = NeuralProphet()
  close_model.fit(open_df, freq="D")

  high_df = X_train[["Date", "High"]]
  high_df.columns = ["ds", "y"]

  high_model = NeuralProphet()
  high_model.fit(open_df, freq="D")

  low_df = X_train[["Date", "Low"]]
  low_df.columns = ["ds", "y"]

  low_model = NeuralProphet()
  low_model.fit(open_df, freq="D")

  return open_model, close_model, high_model, low_model, X_test

def predict(open_model, close_model, high_model, low_model, X_test):
  open_pred_dates = open_model.make_future_dataframe(periods=5) #maybe try switching to true later
  close_pred_dates = close_model.make_future_dataframe(periods=5)
  low_pred_dates = low_model.make_future_dataframe(periods=5)
  high_pred_dates = high_model.make_future_dataframe(periods=5)


  open_predictions = open_model.predict(open_pred_dates)
  close_predictions = close_model.predict(open_pred_dates)
  low_predictions = low_model.predict(open_pred_dates)
  high_predictions = high_model.predict(open_pred_dates)


  print(f'Open Predictions: {open_predictions}')
  print(f'Close Predictions: {close_predictions}')
  print(f'Low Predictions: {low_predictions}')
  print(f'High Predictions: {high_predictions}')

  print(f'Actual Values: {X_test}')


In [84]:
open_model, close_model, high_model, low_model, X_test = train_model(proc_NVIDIA_df)


WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.10/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency B corresponds to 96.169% of the data.
INFO:NP.df_utils:Major frequency B corresponds to 96.169% of the data.
WARNING - (py.warnings._showwarnmsg) - /usr/local/lib/python3.10/dist-packages/neuralprophet/df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

  c

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (40) is too small than the required number                     for the learning rate finder (228). The results might not be optimal.


Finding best initial lr:   0%|          | 0/228 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

WARNING - (NP.config.set_lr_finder_args) - Learning rate finder: The number of batches (40) is too small than the required number                     for the learning rate finder (228). The results might not be optimal.


Finding best initial lr:   0%|          | 0/228 [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

TypeError: NeuralProphet.make_future_dataframe() got an unexpected keyword argument 'include_history'

In [86]:
predict(open_model, close_model, high_model, low_model, X_test)

TypeError: NeuralProphet.make_future_dataframe() missing 1 required positional argument: 'df'